In [1]:
%matplotlib inline

import imageio
import scipy.stats
import scipy.special

import numpy as np
import pandas as pd
from numpy import array as arr

import os
import re
import math

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.image as mpimg
from astropy.io import fits

from scipy.stats import sem 
from scipy import ndimage as ndi
from scipy.optimize import curve_fit

import sys
sys.path.append('../../Analysis_code/klab_python_lib/')

import klib

import time

### Can comment below if not editing klib
%load_ext autoreload
%aimport klib
%autoreload

In [2]:
from typing import Optional
import cv2
from pymba import Vimba, VimbaException, Frame
import time

# todo add more colours
PIXEL_FORMATS_CONVERSIONS = {
    'BayerRG8': cv2.COLOR_BAYER_RG2RGB,
}


def display_frame(frame: Frame, delay: Optional[int] = 1) -> None:
    """
    Displays the acquired frame.
    :param frame: The frame object to display.
    :param delay: Display delay in milliseconds, use 0 for indefinite.
    """
    print('frame {}'.format(frame.data.frameID))

    # get a copy of the frame data
    image = frame.buffer_data_numpy()

    plt.imshow(image)
    plt.show()
#     # convert colour space if desired
#     try:
#         image = cv2.cvtColor(image, PIXEL_FORMATS_CONVERSIONS[frame.pixel_format])
#     except KeyError:
#         pass

#     # display image
#     cv2.imshow('Image', image)
#     cv2.waitKey(delay)

def print_camera_options(cameraID):
    with Vimba() as vimba:
        camera = vimba.camera(cameraID)
        camera.open()

        # get feature value via feature object
        for feature_name in camera.feature_names():
            feature = camera.feature(feature_name)

            try:
                value = feature.value
                range_ = feature.range

                # alternatively the feature value can be read as an object attribute
                # value = getattr(camera, feature_name)
                # or
                # value = camera.someFeatureName

            except VimbaException as e:
                value = e
                range_ = None

            print('\n\t'.join(
                str(x) for x in (
                    feature_name,
                    'value: {}'.format(value),
                    'range: {}'.format(range_))
                if x is not None))

        camera.close()
    
def print_ids():
    with Vimba() as vimba:
        print(vimba.camera_ids())
        
def capture_image(cameraID):
    with Vimba() as vimba:
        camera = vimba.camera(cameraID)
        camera.open()

        camera.feature('TriggerSource').value = 'Software'
        camera.feature('TriggerSelector').value = 'FrameStart'
        camera.feature('AcquisitionFrameRateMode').value = 'Off'
        camera.feature('ExposureMode').value = 'Timed'
        camera.feature('ExposureTime').value = 40
        camera.feature('TriggerMode').value = 'Off'

        camera.arm('SingleFrame')
        # capture a single frame, more than once if desired
        for i in range(1):
            try:
                frame = camera.acquire_frame()
            except VimbaException as e:
                # rearm camera upon frame timeout
                if e.error_code == VimbaException.ERR_TIMEOUT:
                    print(e)
                    camera.disarm()
                    camera.arm('SingleFrame')
                else:
                    raise
        image = frame.buffer_data_numpy()
        camera.disarm()
        camera.close()
        np.savetxt('tmp_img.txt', image, fmt='%d')
#         plt.imshow(image)

In [19]:
print_ids()
print_camera_options(0)

['DEV_1AB2280006D0']
AcquisitionAbort
	value: Cannot get or set the value of a command feature type, call the command instead.
	range: None
AcquisitionFrameCount
	value: Operation is invalid with the current access mode.
	range: None
AcquisitionFrameRate
	value: 14.04905454990784
	range: (0.0002777777809382716, 14.04905454990784)
AcquisitionFrameRateMode
	value: Off
	range: ['Off', 'Basic']
AcquisitionMode
	value: SingleFrame
	range: ['SingleFrame', 'MultiFrame', 'Continuous']
AcquisitionStart
	value: Cannot get or set the value of a command feature type, call the command instead.
	range: None
AcquisitionStatus
	value: False
	range: None
AcquisitionStatusSelector
	value: AcquisitionTriggerWait
	range: ['AcquisitionTriggerWait', 'AcquisitionActive', 'AcquisitionTransfer', 'FrameTriggerWait', 'FrameActive', 'ExposureActive']
AcquisitionStop
	value: Cannot get or set the value of a command feature type, call the command instead.
	range: None
BlackLevel
	value: 7.0
	range: (7.0, 63.0)
Blac

In [35]:
with Vimba() as vimba:
    camera = vimba.camera(0)
    camera.open()

    camera.feature('TriggerSource').value = 'Software'
    camera.feature('TriggerSelector').value = 'FrameStart'
    camera.feature('AcquisitionFrameRateMode').value = 'Off'
    camera.feature('ExposureMode').value = 'Timed'
    camera.feature('ExposureTime').value = 40
    camera.feature('TriggerMode').value = 'Off'

    start = time.time()

    frames = []
    camera.arm('SingleFrame')
    # capture a single frame, more than once if desired
    for i in range(10):
        try:
            frame = camera.acquire_frame()
            frames.append(frame)
        except VimbaException as e:
            # rearm camera upon frame timeout
            if e.error_code == VimbaException.ERR_TIMEOUT:
                print(e)
                camera.disarm()
                camera.arm('SingleFrame')
            else:
                raise    
    end = time.time()
    camera.disarm()
    camera.close()

print(end-start)

2.3790364265441895


In [61]:
image = frames[0].buffer_data_numpy()
plt.imshow(np.abs(np.log(image))